# Segmenting and Clustering Neighborhoods in Toronto (Week 3)

by Robert

In this assignment, we explore, segment, and cluster the neighborhoods in the city of Toronto based on the postal code and borough information.

Install and import all the libraries and modules that we will need:

In [1]:
!pip install geopy

In [2]:
!pip install folium==0.5.0

In [3]:
!pip install beautifulsoup4

In [4]:
# library for web parsing
from bs4 import BeautifulSoup

# library to handle data in a vectorized manner
import numpy as np

# library for data analysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

# library to handle requests
import requests

# module for converting an address into latitude and longitude values
from geopy.geocoders import Nominatim

# module for clustering
from sklearn.cluster import KMeans

#  modules for plotting clusters
import matplotlib.cm as cm
import matplotlib.colors as colors

# library for plotting maps
import folium

## 1. Web Scraping

First, we download the HTML file (the web page) from the URL:

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_data = requests.get(url).text

Create a <code>BeautifulSoup</code> object, extract the information we need from it, and store the information all in one list:

In [6]:
soup = BeautifulSoup(raw_data, 'html.parser')
store_list = []

for row in soup.tbody.find_all('tr'):
    col = row.find_all('td')
    if len(col) == 0:
        continue
    else:
        pc = str(col[0].get_text()).strip()
        borough_name = str(col[1].get_text()).strip()
        neighborhood_name = str(col[2].get_text()).strip()
    
        if borough_name == 'Not assigned':
            continue
        elif neighborhood_name == 'Not assigned':
            neighborhood_name = borough_name
    
        store_list.append((pc, borough_name, neighborhood_name))

print(store_list[:10])

[('M3A', 'North York', 'Parkwoods'), ('M4A', 'North York', 'Victoria Village'), ('M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'), ('M6A', 'North York', 'Lawrence Manor, Lawrence Heights'), ('M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"), ('M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'), ('M1B', 'Scarborough', 'Malvern, Rouge'), ('M3B', 'North York', 'Don Mills'), ('M4B', 'East York', 'Parkview Hill, Woodbine Gardens'), ('M5B', 'Downtown Toronto', 'Garden District, Ryerson')]


The list is a list of tuples. We can directly convert it into a Pandas dataframe:

In [7]:
columns = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(data=store_list, columns=columns)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check how many rows are in this dataframe:

In [8]:
df.shape

(103, 3)

## 2. Getting Coordinates

Now we have to get the latitude and longitude coordinates of each postal code, and add them to the corresponding boroughs/neighborhoods.

Actually, Canadian postal codes are by definition alphanumeric and each postal code contains six characters, of which the first three -- same to what we see here in the dataframe -- represent a [Forward Sortation Area (FSA)](https://www.ic.gc.ca/eic/site/bsf-osb.nsf/eng/br03396.html).

For a stabler processing, we download the CSV file from the source, and read it into a dataframe:

In [9]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

In [10]:
coord_df = pd.read_csv('Geospatial_Coordinates.csv')
coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then we merge (inner join) the two dataframes we have at hand right now, in exchange for a single dataframe containing postal codes, borough names, neighborhood names, latitudes and longitudes:

In [11]:
coord_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
neighborhoods_df = df.merge(coord_df, how='inner', on='PostalCode')
neighborhoods_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 3. Exploring and Clustering

In this practice, to save processing time, we work with only boroughs that contain the word "Toronto". 

### 3.1. Filtering the Data

We first filter out these borough by selecting rows using <code>pandas.Series.str.contains()</code> method:

In [12]:
targets_df = neighborhoods_df[neighborhoods_df['Borough'].str.contains('Toronto')]
targets_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


Let's see how many distinct postal codes (i.e., FSAs) and distinct boroughs there are.

In [13]:
print('There are {} distinct postal codes (forward sortation areas) to explore'.format(targets_df['PostalCode'].nunique()))
print('There are {} distinct boroughs to explore'.format(targets_df['Borough'].nunique()))

There are 40 distinct postal codes (forward sortation areas) to explore
There are 5 distinct boroughs to explore


**Since one borough can contain one or more FSAs and each FSA (instead of borough) here has its own latitude and longitude coordinates, we will do our exploring and clustering by FSA instead of by borough.**

### 3.2. Exploring the FSAs

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods.

Load first the credentials for the API:

In [14]:
# Define your credentials
CLIENT_ID = 'WCB2GYQ5IIJN11ELWJM5O5BJA22CCA2W2VT2SC5XQYWAJSXL'
CLIENT_SECRET = '3V3IPQ41NEARKE5DQYXLKTKN4XVMYRIMFPQVFX4FKNZPYYTQ'
ACCESS_TOKEN = 'XSCZMBKGRGKASXTMR005JSIKP5NIYZZIXFALHJ0HN2EWP1JO'
VERSION = '20180604'

We define a function, <code>get_category_type</code>, to extract the explored venues' category type:

In [15]:
# Define a function that extracts the category of the venue
def get_category_type(row):
    categories_array = row['categories'] # returns the cell value
    
    if len(categories_array) == 0:
        return None
    else:
        return categories_array[0]['name']

Also, we define another function, <code>getNearbyVenues</code>, to batch-process the explorations using Foursquare API:

In [16]:
def getNearbyVenues(postalcodes, latitudes, longitudes, radius=500, limit=100):
    
    venues_list = list()
    # Zip the iterables
    for postalcode, lat, lng in zip(postalcodes, latitudes, longitudes):
        
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Extract the wanted part of JSON and extend the list tuple-by-tuple
        # The list becomes a list of tuples after extraction and extension
        venues_list.extend([(
            postalcode,
            lat, 
            lng, 
            each['venue']['name'], 
            each['venue']['location']['lat'], 
            each['venue']['location']['lng'],  
            each['venue']['categories'][0]['name']) for each in results])
    
    # Transform the list of tuples into a dataframe -- it can be directly converted by pandas.DataFrame()
    # Also see pandas.DataFrame.from_records()
    nearby_venues = pd.DataFrame(venues_list)
    nearby_venues.columns = ['PostalCode', 
                  'FSA Latitude', 
                  'FSA Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Apply the function and store the all exploration results in a new dataframe:

In [17]:
explore_results_df = getNearbyVenues(postalcodes=targets_df['PostalCode'], latitudes=targets_df['Latitude'], longitudes=targets_df['Longitude'])

Take a look at the results:

In [18]:
explore_results_df.head()

,PostalCode,FSA Latitude,FSA Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


We merge the <code>explore_results_df</code> dataframe with the previous dataframe <code>targets_df</code> that stores borough names containing "Toronto" and corresponding neighborhood names, in order to make the former more human-readable:

In [19]:
explore_results_df = explore_results_df.merge(targets_df, how='inner', on='PostalCode')

# Drop excessive columns
explore_results_df.drop(['Latitude', 'Longitude'], axis=1, inplace=True) 
explore_results_df.head()

,PostalCode,FSA Latitude,FSA Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighborhood
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,Downtown Toronto,"Regent Park, Harbourfront"
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,Downtown Toronto,"Regent Park, Harbourfront"
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,Downtown Toronto,"Regent Park, Harbourfront"
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,Downtown Toronto,"Regent Park, Harbourfront"
4,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant,Downtown Toronto,"Regent Park, Harbourfront"


### 3.3. Analysing each FSA

We convert each venue category into numerical values (one-hot encoding) for later steps:

In [20]:
# Label the categories by one-hot encoding for later clustering
venue_onehot_df = pd.get_dummies(explore_results_df[['Venue Category']], prefix="", prefix_sep="")
venue_onehot_df.head()

,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [21]:
# Add back necessary columns to the new dataframe
venue_onehot_df.insert(0, 'Neighborhood Names', explore_results_df['Neighborhood'])
venue_onehot_df.insert(0, 'Borough', explore_results_df['Borough'])
venue_onehot_df.insert(0, 'FSA', explore_results_df['PostalCode'])
venue_onehot_df.head()

,FSA,Borough,Neighborhood Names,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

For later clustering, we now group rows by neighborhood, and see the mean of the frequency (occurrence) of each category. This mean value can be regarded as a normalized score for a specific category's "liveliness" within each FSA.

In [22]:
venue_grouped_df = venue_onehot_df.groupby(['FSA', 'Borough', 'Neighborhood Names']).mean().reset_index()
venue_grouped_df.head()

,FSA,Borough,Neighborhood Names,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,

We now need to get the top 5 venue categories of each distinct FSA.

For batch-processing, we define another function <code>return_most_common_venues</code>:

In [23]:
def return_most_common_venues(row, num_top_venues):
    # Chop off each row's non-numerical attributes at row[0:3]
    row_categories = row.iloc[3:]
    # Sort the series in descending order by frequency
    # The series contains only category names (as index) and corresponding frequency scores (as values)
    row_categories_sorted = row_categories.sort_values(ascending=False)
    # Return nth top category names in ndarray form
    return row_categories_sorted.index.values[0:num_top_venues]

Then create a new dataframe, get the top 5 venue categories for each distinct FSA, and store them in the new dataframe:


In [24]:
num_top_venues = 5

# Create columns list according to number of top n venue categories
columns = ['FSA', 'Borough', 'Neighborhood Names']
indicators = ['st', 'nd', 'rd']
for n in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(n+1, indicators[n]))
    except:
        columns.append('{}th Most Common Venue'.format(n+1))

# Create a new dataframe to store
top_venues_df = pd.DataFrame(columns=columns)
top_venues_df['FSA'] = venue_grouped_df['FSA']
top_venues_df['Borough'] = venue_grouped_df['Borough']
top_venues_df['Neighborhood Names'] = venue_grouped_df['Neighborhood Names']

# Put the nth top category names (in ndarray form) of each FSA in 
for i in np.arange(venue_grouped_df.shape[0]):
    top_venues_df.iloc[i, 3:] = return_most_common_venues(venue_grouped_df.iloc[i, :], num_top_venues)

top_venues_df.head()

,FSA,Borough,Neighborhood Names,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Sandwich Place,Fast Food Restaurant,Pub,Liquor Store
3,M4M,East Toronto,Studio District,Coffee Shop,American Restaurant,Bakery,Brewery,Café
4,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Business Service,Swim School,Discount Store


### 3.4. Clustering

We run <code>sklearn.cluster.KMeans</code> to cluster the neighborhood into 5 clusters.

In [25]:
k = 5
# Chop off non-numerical columns
feed_clustering_df = venue_grouped_df.drop(['FSA', 'Borough', 'Neighborhood Names'], axis=1)

# Fit the model
kmeans_model = KMeans(n_clusters=k, random_state=0).fit(feed_clustering_df)
kmeans_model.labels_[:10]

array([3, 3, 3, 3, 0, 3, 3, 3, 1, 3], dtype=int32)

Insert the cluster labels to the <code>top_venues_df</code> dataframe:

In [26]:
top_venues_df.insert(0, 'Cluster Labels', kmeans_model.labels_)
top_venues_df.head()

,Cluster Labels,FSA,Borough,Neighborhood Names,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,3,M4E,East Toronto,The Beaches,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio
1,3,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop
2,3,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Sandwich Place,Fast Food Restaurant,Pub,Liquor Store
3,3,M4M,East Toronto,Studio District,Coffee Shop,American Restaurant,Bakery,Brewery,Café
4,0,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Business Service,Swim School,Discount Store


We add back each FSA's latitude and longitude coordinates for later visualization:

In [27]:
add_back_df = targets_df[['PostalCode', 'Latitude', 'Longitude']]
add_back_df.columns = ['FSA', 'FSA Latitude', 'FSA Longitude']
top_venues_df = top_venues_df.merge(add_back_df, how='inner', on='FSA')
rearrange_columns = list(top_venues_df.columns[-2:]) + list(top_venues_df.columns[:-2])
top_venues_df = top_venues_df[rearrange_columns]
top_venues_df.head()

,FSA Latitude,FSA Longitude,Cluster Labels,FSA,Borough,Neighborhood Names,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,43.676357,-79.293031,3,M4E,East Toronto,The Beaches,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio
1,43.679557,-79.352188,3,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop
2,43.668999,-79.315572,3,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Sandwich Place,Fast Food Restaurant,Pub,Liquor Store
3,43.659526,-79.340923,3,M4M,East Toronto,Studio District,Coffee Shop,American Restaurant,Bakery,Brewery,Café
4,43.728020,-79.388790,0,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Business Service,Swim School,Discount Store


### 3.5. Visualization

We plot the clusters on a map using Folium:

In [28]:
# Generate a map centered around Toronto
latitude = 43.6532
longitude =  -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# Set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, fsa, borough, cluster in zip(top_venues_df['FSA Latitude'], top_venues_df['FSA Longitude'], top_venues_df['FSA'], top_venues_df['Borough'], top_venues_df['Cluster Labels']):
    label = folium.Popup(str(fsa) + '; Borough' + '; Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_clusters)
       
map_clusters

### 3.6. Examining Clusters

We try to examine each cluster and determine the discriminating venue categories that distinguish each cluster.

**Cluster 1**

In [29]:
top_venues_df.loc[top_venues_df['Cluster Labels'] == 0, top_venues_df.columns[3:]]

,FSA,Borough,Neighborhood Names,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Business Service,Swim School,Discount Store
10,M4W,Downtown Toronto,Rosedale,Park,Playground,Trail,Dessert Shop,Event Space


**Cluster 2**

In [30]:
top_venues_df.loc[top_venues_df['Cluster Labels'] == 1, top_venues_df.columns[3:]]

,FSA,Borough,Neighborhood Names,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,M4T,Central Toronto,"Moore Park, Summerhill East",Park,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant


**Cluster 3**

In [31]:
top_venues_df.loc[top_venues_df['Cluster Labels'] == 2, top_venues_df.columns[3:]]

,FSA,Borough,Neighborhood Names,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
34,M6N,Toronto/York,"Runnymede, The Junction, Weston-Pellam Park, C...",Grocery Store,Breakfast Spot,Bus Line,Caribbean Restaurant,Discount Store


**Cluster 4**

In [32]:
top_venues_df.loc[top_venues_df['Cluster Labels'] == 3, top_venues_df.columns[3:]]

,FSA,Borough,Neighborhood Names,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Sandwich Place,Fast Food Restaurant,Pub,Liquor Store
3,M4M,East Toronto,Studio District,Coffee Shop,American Restaurant,Bakery,Brewery,Café
5,M4P,Central Toronto,Davisville North,Hotel,Gym / Fitness Center,Pizza Place,Department Store,Sandwich Place
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Salon / Barbershop
7,M4S,Central Toronto,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Café,Italian Restaurant
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pizza Place,Liquor Store,Restaurant,Pub
11,M4X,Downtown Toronto,"St. James Town, Cabbagetown",Coffee Shop,Italian Restaurant,Bakery,Restaurant,Pizza Place
12,M4Y,Downtown Toronto,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar


**Cluster 5**

In [33]:
top_venues_df.loc[top_venues_df['Cluster Labels'] == 4, top_venues_df.columns[3:]]

,FSA,Borough,Neighborhood Names,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,M5N,Central Toronto,Roselawn,Home Service,Pool,Garden,Yoga Studio,Dessert Shop
